In [1]:
from google.colab import drive
# INGET PAKE STD STEI
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!python3 --version

Python 3.8.10


In [3]:
# !pip install tensorflow-gpu==1.15.0

In [4]:
import numpy as np
import os
import pandas as pd

In [5]:
# path_to_data='/content/drive/MyDrive/code/Data_Train/'
# os.listdir(path_to_data)###

In [6]:
# print (os.listdir(path_to_data))

In [7]:
##clone xlnet official repository
!git clone https://github.com/arfaghif/malaya

Cloning into 'malaya'...
remote: Enumerating objects: 17730, done.
remote: Counting objects: 100% (17730/17730), done.
remote: Compressing objects: 100% (4520/4520), done.
remote: Total 17730 (delta 12822), reused 17712 (delta 12814), pack-reused 0
Receiving objects: 100% (17730/17730), 121.98 MiB | 16.62 MiB/s, done.
Resolving deltas: 100% (12822/12822), done.


### Using sentence piece model from Google to tokenize text
We will be using  Google's **sentencepiece** model to tokenize the text. 

In [8]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00


In [9]:
# path_to_data

In [10]:
curr_dir=os.getcwd()

In [11]:
curr_dir

'/content'

In [12]:
%cd '/content/drive/MyDrive/code/Finetuning/new'

/content/drive/MyDrive/code/Finetuning/new


In [13]:
!ls

airy_car_corpus.txt   all_corpus.txt		car_corpus.txt
airy_corpus.txt       all_sp10m.cased.v3.model	car_rest_corpus.txt
airy_rest_corpus.txt  all_sp10m.cased.v3.vocab	rest_corpus.txt


We will be keeping vocab_size to 12000 and unigram model.

In [14]:
# # spm_train --input=data_lm.txt --model_prefix=sp10m.cased.v3 
# # --vocab_size=12000 --character_coverage=0.99995 --model_type=unigram
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=airy_corpus.txt  --vocab_size=10175 --model_prefix=all_sp10m.cased.v3 --character_coverage=0.99995 --model_type=unigram')


In [ ]:
os.listdir(path_to_data)##now sentence piece model has been saved

In [ ]:
%cd '/content'

In [ ]:
%cd '/content/malaya/pretrained-model/xlnet'

In [ ]:
print (os.listdir(path_to_data + "corpus/"))

In [ ]:
##We will be saving tf records in a folder named experimenttfrecords

In [ ]:
%%writefile prepro_utils.py
# coding=utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import unicodedata
import six
from functools import partial


SPIECE_UNDERLINE = '▁'


def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def print_(*args):
  new_args = []
  for arg in args:
    if isinstance(arg, list):
      s = [printable_text(i) for i in arg]
      s = ' '.join(s)
      new_args.append(s)
    else:
      new_args.append(printable_text(arg))
  print(*new_args)


def preprocess_text(inputs, lower=False, remove_space=True, keep_accents=False):
  if remove_space:
    outputs = ' '.join(inputs.strip().split())
  else:
    outputs = inputs
  outputs = outputs.replace("``", '"').replace("''", '"')

  if six.PY2 and isinstance(outputs, str):
    outputs = outputs.decode('utf-8')

  if not keep_accents:
    outputs = unicodedata.normalize('NFKD', outputs)
    outputs = ''.join([c for c in outputs if not unicodedata.combining(c)])
  if lower:
    outputs = outputs.lower()

  return outputs


def encode_pieces(sp_model, text, return_unicode=True, sample=False):
  # return_unicode is used only for py2

  # note(zhiliny): in some systems, sentencepiece only accepts str for py2
  if six.PY2 and isinstance(text, unicode):
    text = text.encode('utf-8')

  if not sample:
    pieces = sp_model.EncodeAsPieces(text)
  else:
    pieces = sp_model.SampleEncodeAsPieces(text, 64, 0.1)
  new_pieces = []
  for piece in pieces:
    if len(piece) > 1 and piece[-1] == ',' and piece[-2].isdigit():
      cur_pieces = sp_model.EncodeAsPieces(
          piece[:-1].replace(SPIECE_UNDERLINE, ''))
      if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
        if len(cur_pieces[0]) == 1:
          cur_pieces = cur_pieces[1:]
        else:
          cur_pieces[0] = cur_pieces[0][1:]
      cur_pieces.append(piece[-1])
      new_pieces.extend(cur_pieces)
    else:
      new_pieces.append(piece)

  # note(zhiliny): convert back to unicode for py2
  if six.PY2 and return_unicode:
    ret_pieces = []
    for piece in new_pieces:
      if isinstance(piece, str):
        piece = piece.decode('utf-8')
      ret_pieces.append(piece)
    new_pieces = ret_pieces

  return new_pieces


def encode_ids(sp_model, text, sample=False):
  pieces = encode_pieces(sp_model, text, return_unicode=False, sample=sample)
  ids = [sp_model.PieceToId(piece) for piece in pieces]
  return ids


if __name__ == '__main__':
  import sentencepiece as spm

  sp = spm.SentencePieceProcessor()
  sp.load('sp10m.uncased.v3.model')

  print_(u'I was born in 2000, and this is falsé.')
  print_(u'ORIGINAL', sp.EncodeAsPieces(u'I was born in 2000, and this is falsé.'))
  print_(u'OURS', encode_pieces(sp, u'I was born in 2000, and this is falsé.'))
  print(encode_ids(sp, u'I was born in 2000, and this is falsé.'))
  print_('')
  prepro_func = partial(preprocess_text, lower=True)
  print_(prepro_func('I was born in 2000, and this is falsé.'))
  print_('ORIGINAL', sp.EncodeAsPieces(prepro_func('I was born in 2000, and this is falsé.')))
  print_('OURS', encode_pieces(sp, prepro_func('I was born in 2000, and this is falsé.')))
  print(encode_ids(sp, prepro_func('I was born in 2000, and this is falsé.')))
  print_('')
  print_('I was born in 2000, and this is falsé.')
  print_('ORIGINAL', sp.EncodeAsPieces('I was born in 2000, and this is falsé.'))
  print_('OURS', encode_pieces(sp, 'I was born in 2000, and this is falsé.'))
  print(encode_ids(sp, 'I was born in 2000, and this is falsé.'))
  print_('')
  print_('I was born in 92000, and this is falsé.')
  print_('ORIGINAL', sp.EncodeAsPieces('I was born in 92000, and this is falsé.'))
  print_('OURS', encode_pieces(sp, 'I was born in 92000, and this is falsé.'))
  print(encode_ids(sp, 'I was born in 92000, and this is falsé.'))

In [ ]:
# !git pull

In [ ]:
!python data_utils.py \
   --use_tpu=False \
   --bsz_per_host=8 \
   --num_core_per_host=1 \
   --uncased=False \
   --seq_len=128 \
   --reuse_len=64 \
   --input_glob="/content/drive/MyDrive/code/Data_Train/corpus/airy_corpus.txt" \
   --save_dir="/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune" \
   --sp_path="/content/drive/MyDrive/code/Data_Train/corpus/sp10m.cased.v3.model" \
   --mask_alpha=6 --mask_beta=1 --num_predict=85 --bi_data=False

Now, it's time to train the model.

In [ ]:
os.getcwd()

In [ ]:
%cd /content/malaya/pretrained-model/xlnet

In [ ]:
# !wget https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/xlnet-base-29-03-2020.tar.gz

In [ ]:
# !tar -xvf xlnet-base-29-03-2020.tar.gz

###  We will be training our language model from checkpoint. Checkpoint is the model downloaded above.

We will save the newly trained model inside the directoy experimenttfrecords created above. If we omit init_checkpoint argument we will get freshly trained language model.

In [ ]:
# !cp -r 'xlnet-base-29-03-2020' '/content/drive/My Drive/code'

In [ ]:
# !git pull

In [ ]:
# !wget https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip

In [ ]:
# !unzip cased_L-12_H-768_A-12.zip

In [ ]:
!python train_gpu.py \
   --corpus_info_path="/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/corpus_info.json" \
   --model_dir="/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/model"  \
   --record_info_dir="/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/tfrecords" \
   --train_batch_size=8 \
   --seq_len=128 \
   --reuse_len=64 \ 
   --mem_len=384 \
   --perm_size=256 \
   --untie_r=True \
   --mask_alpha=6 \
   --mask_beta=1 \
   --num_predict=85 \
   --train_steps=100 \
   --iterations=2 \
   --uncased=False \
   --bi_data=False \
   --save_steps=2 \
   --d_head=64 \
   --d_inner=3072 \
   --d_model=768 \
   --ff_activation=gelu \
   --n_head=12 \
   --n_layer=12 \
   --n_token=32000 \
   --init_checkpoint='/content/drive/My Drive/code/xlnet-base-29-03-2020/model.ckpt-300000'

In [ ]:
# !python3 train_gpu.py \
#   --corpus_info_path="/kaggle/working/experimenttfrecords/corpus_info.json" \
#   --record_info_dir="/content/drive/My Drive/experimenttfrecords/tfrecords" \
#   --train_batch_size=20 \
#   --seq_len=512 \
#   --reuse_len=256 \
#   --mem_len=384 \
#   --perm_size=256 \
#   --n_layer=12 \
#   --d_model=768 \
#   --d_embed=768 \
#   --n_head=12 \
#   --d_head=64 \
#   --d_inner=3072 \
#   --untie_r=True \
#   --mask_alpha=6 \
#   --mask_beta=1 \
#   --num_predict=85 \
#   --model_dir=output-model \
#   --uncased=False \
#   --num_core_per_host=1 \
#   --train_steps=20000  --iterations=10 --learning_rate=5e-5 \
#   --num_gpu_cores=2

In [ ]:
%cd '/content/'

In [ ]:
#"/content/drive/MyDrive/TA/code/finetune/experimenttfrecords/model

In [ ]:
!git clone https://github.com/huggingface/transformers

Converting tensorflow model into pytorch model

In [ ]:
# export TRANSFO_XL_CHECKPOINT_PATH='base_cased/model.ckpt'
# export TRANSFO_XL_CONFIG_PATH='base_cased/config.json '

# python transformers-cli convert --model_type xlnet  --tf_checkpoint $TRANSFO_XL_CHECKPOINT_PATH --config $TRANSFO_XL_CONFIG_PATH --pytorch_dump_output xlnet_exp_pytorch


In [ ]:
%cd '/content/transformers/'

In [ ]:
!export TRANSFO_XL_CHECKPOINT_PATH='/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/model/model.ckpt'
!export TRANSFO_XL_CONFIG_PATH='/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/model/config.json'
!export PYTORCH_MODELPATH='/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/pytorch_model'

In [ ]:
!pip install transformers

In [ ]:
!python3 transformers-cli convert \
    --model_type xlnet  \
    --tf_checkpoint="/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/model/model.ckpt" \
    --config="/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/model/config.json" \
    --pytorch_dump_output='/content/drive/MyDrive/TA/code/finetune/airy_corpus_finetune/pytorch_model'